In [2]:
import pandas as pd
data_path = "commodities_12_22.csv"
# Read the CSV file into a pandas dataframe
df = pd.read_csv(data_path)

# This is for EDA

In [20]:
# Print the first few rows of the dataframe (optional)
dtypes = df.dtypes
print(f'''{df.head()}

Data Types of each column: \n{dtypes}''')


        Date  Crude Oil  Brent Oil  Natural Gas    Gold  Silver  Copper
0 2022-06-15     119.11     121.35        7.296  1814.8  21.090  4.1777
1 2022-06-14     118.93     121.17        7.189  1813.5  20.954  4.1565
2 2022-06-13     120.93     122.27        8.609  1831.8  21.255  4.2115
3 2022-06-10     120.67     122.01        8.850  1875.5  21.931  4.2945
4 2022-06-09     121.51     123.07        8.963  1852.8  21.817  4.3810

Data Types of each column: 
Date           datetime64[ns]
Crude Oil             float64
Brent Oil             float64
Natural Gas           float64
Gold                  float64
Silver                float64
Copper                float64
dtype: object


In [21]:
print(df)

           Date  Crude Oil  Brent Oil  Natural Gas    Gold  Silver  Copper
0    2022-06-15     119.11     121.35        7.296  1814.8  21.090  4.1777
1    2022-06-14     118.93     121.17        7.189  1813.5  20.954  4.1565
2    2022-06-13     120.93     122.27        8.609  1831.8  21.255  4.2115
3    2022-06-10     120.67     122.01        8.850  1875.5  21.931  4.2945
4    2022-06-09     121.51     123.07        8.963  1852.8  21.817  4.3810
...         ...        ...        ...          ...     ...     ...     ...
3254 2012-01-29        NaN        NaN          NaN     NaN  33.908     NaN
3255 2012-01-22        NaN        NaN          NaN     NaN  31.793     NaN
3256 2012-01-15        NaN        NaN          NaN     NaN  29.785     NaN
3257 2012-01-08        NaN        NaN          NaN     NaN  28.848     NaN
3258 2012-01-02        NaN        NaN          NaN     NaN  28.160     NaN

[3259 rows x 7 columns]


In [24]:
#Total Rows
totalRows = df.shape[0]

#Counting the # of Nulls
nulls = df.isnull().sum(axis=1)
rowCountWNulls = len(nulls[nulls>0])

#Latest null row
df['Date'] = pd.to_datetime(df['Date'])
#Find the first row with a Null value (Excluding 'Date')

nullMask = df.iloc[:, 1:].isnull().any(axis = 1) #Check for nulls except for Date
latestNullRow = df[nullMask].iloc[0]

print(f'''Total rows: {totalRows}
Number of rows with nulls: {rowCountWNulls}
Null Date: {latestNullRow.iloc[0]}''')
#Latest Date w/ a Null: {latestNullRow}

Total rows: 3259
Number of rows with nulls: 603
Null Date: 2022-05-29 00:00:00
